# Обработка данных для команды игры "Секреты Темнолесья"

- Автор: Василенко Егор
- Дата: 22.12.2024

### Цели и задачи проекта

- Отобрать данные по времени выхода игры за период с 2000 по 2013 год включительно.
- Категоризовать игры по оценкам пользователей и экспертов. Выделите три категории:
  - высокая оценка — с оценкой от 8 до 10 и от 80 до 100, включая правые границы интервалов.
  - средняя оценка — с оценкой от 3 до 8 и от 30 до 80, не включая правые границы интервалов.
  - низкая оценка — с оценкой от 0 до 3 и от 0 до 30, не включая правые границы интервалов.
- Выделить топ-7 платформ по количеству игр, выпущенных за весь требуемый период.

### Описание данных

Данные `/datasets/new_games.csv` содержат информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр:
- `Name` — название игры.
- `Platform` — название платформы.
- `Year of Release` — год выпуска игры.
- `Genre` — жанр игры.
- `NA sales` — продажи в Северной Америке (в миллионах проданных копий).
- `EU sales` — продажи в Европе (в миллионах проданных копий).
- `JP sales` — продажи в Японии (в миллионах проданных копий).
- `Other sales` — продажи в других странах (в миллионах проданных копий).
- `Critic Score` — оценка критиков (от 0 до 100).
- `User Score` — оценка пользователей (от 0 до 10).
- `Rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

<a class="anchor" id="Содержание"></a>
### Содержимое проекта

- [Загрузка данных и знакомство с ними](#Знакомство-с-данными)
- [Обработка данных](#Обработка-данных)
  - [Работа с типом данных | Часть 1](#Работа-с-типом-данных-ч1)
  - [Работа с пропусками](#Работа-с-пропусками)
  - [Работа с типом данных | Часть 2](#Работа-с-типом-данных-ч2)
  - [Работа с дубликатами](#Работа-с-дубликатами)
- [Фильтрация данных](#Фильтрация-данных)
- [Категоризация данных](#Категоризация-данных)
- [Итоговый вывод](#Итоговый-вывод)
---

<a class="anchor" id="Знакомство-с-данными"></a>
## 1. Загрузка данных и знакомство с ними

### Загрузка библиотек

In [7]:
# Загрузка библиотеки pandas
import pandas as pd

### Загрузка данных

In [9]:
# Загрузка файла формата .csv
data = pd.read_csv('/datasets/new_games.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/new_games.csv'

In [ ]:
# Создание копии для обхода предупреждения, которое не помогло избежать всех предупреждений
df = data.copy()

### Знакомство с данными

- Познакомьтесь с данными: выведите первые строки и результат метода `info()`.


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
total_rows_1 = df.shape[0]

- Сделайте вывод о полученных данных: данные какого объёма вам предоставили, соответствуют ли они описанию, встречаются ли в них пропуски, используются ли верные типы данных.
- Отметьте другие особенности данных, которые вы обнаружили и на которые стоит обратить внимание при предобработке. Например, вы можете проверить названия столбцов: все ли названия отражают содержимое данных и прописаны в удобном для работы виде.

### Выводы по данным

- Всего в датасете 16 956 строк и 11 столбцов.
- Предоставленные данные соответствуют описанию, однако в части из них используются неверные типы данных:
  - `Year of release` — нужен тип данных `datetime`
  - `EU sales` — нужен тип данных `float`
  - `JP sales` — нужен тип данных `float`
  - `User Score` — нужен тип данных `float`
  - `Rating` — нужен тип данных `float`
  - `Critic score` — нужен тип данных `int` (на metacritic оценки целые, а это наверное оно)
- В 6-ти столбцах датасета встречаются пропуски.
- Названия столбцов стоит привести к единому стилу в соответствии с ТЗ, однако они отражают содержимое данных.

[Вернуться к содержанию](#Содержание)

<font color='DarkBlue'><b>Комментарий ревьюера</b></font><br>
<font color='DarkGreen'>👌 Хорошо, исследовали общую информацию о датасетах.</font>

---
<a class="anchor" id="Обработка-данных"></a>
## 2.  Проверка ошибок в данных и их предобработка


### 2.1. Названия, или метки, столбцов датафрейма

- Выведите на экран названия всех столбцов датафрейма и проверьте их стиль написания.
- Приведите все столбцы к стилю snake case. Названия должны быть в нижнем регистре, а вместо пробелов — подчёркивания.

In [ ]:
df.columns

In [ ]:
# Приведение названий столбцов к формату snake_case
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [ ]:
df.columns

<a class="anchor" id="Работа-с-типом-данных-ч1"></a>
### 2.2. Типы данных

- Если встречаются некорректные типы данных, предположите их причины.
- При необходимости проведите преобразование типов данных. Помните, что столбцы с числовыми данными и пропусками нельзя преобразовать к типу `int64`. Сначала вам понадобится обработать пропуски, а затем преобразовать типы данных.

In [ ]:
# Комментарий ревьюера
tmp = df.copy() # создаем копию датасета до преобразования
len(tmp)

In [ ]:
df.dtypes

- В числовых столбцах могут встретиться строковые значения, например `unknown` или другие. Приводите такие столбцы к числовому типу данных, заменив строковые значения на пропуски.

Числовые данные, имеющие не числовой тип данных:
- `user_score`
- `jp_sales`
- `eu_sales`

In [ ]:
for col in ['user_score', 'jp_sales', 'eu_sales']:
    display(df[col].unique())

**Вывод по анализу:**
- Столбцы `user_score`, `jp_score`, `eu_sales` имеют тип данных `object`, что обусловлено наличием значения строкового типа `unknown` в столбцах `jp_score`, `eu_sales` и значения строкового типа `tbd` в столбце `user_score`. Именно данный тип данных позволяет хранить и числовые, и нечисловые данные.
- Вероятно, ошибка в столбцах `jp_score`, `eu_sales`связаны с ошибков в данных или неправильной записью.
- Ошибка же в `user_score` же наиболее вероятно является ошибкой заполнения данных, ибо `tbd` - это `to be determined`, то есть подлежит уточнению.

In [ ]:
# Приведение типов данных в продажам по Европе и Японии к float64 с заменой строк на NaN через аргумент errors = 'coerce'
df['eu_sales'] = pd.to_numeric(df['eu_sales'], errors='coerce')
df['jp_sales'] = pd.to_numeric(df['jp_sales'], errors='coerce')
df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce')

In [ ]:
df.dtypes

[Вернуться к содержанию](#Содержание)

<a class="anchor" id="Работа-с-пропусками"></a>
### 2.3. Наличие пропусков в данных

- Посчитайте количество пропусков в каждом столбце в абсолютных и относительных значениях.


In [ ]:
print(f"Количество пропусков в абсолютных значениях:")
display(pd.DataFrame(df.isna().sum(), columns=['Total NaN'])
        .style.background_gradient('coolwarm'))

In [ ]:
print(f"Количество пропусков в относительных значениях (%):")
display(pd.DataFrame(round((df.isna().mean()*100),2), columns=['NaNs, %'])
        .style.format( '{:.2f}').background_gradient('coolwarm'))

In [ ]:
# Комментарий ревьюера
temp = df.copy()
display(pd.DataFrame(round((temp.isna().mean()*100),2), columns=['NaNs, %'])
        .style.format( '{:.2f}').background_gradient('coolwarm'))

In [ ]:
df[df[['eu_sales', 'jp_sales']].isna().any(axis = 1)]

- Изучите данные с пропущенными значениями. Напишите промежуточный вывод: для каких столбцов характерны пропуски и сколько их. Предположите, почему пропуски могли возникнуть. Укажите, какие действия с этими данными можно сделать и почему.


**Выводы о пропусках**

--------------------
- Незначительное количество пропусков присутствует в 5 столбцах из 11, в 3 из 11 значительное количество. В совокупности пропуски в 8 из 11 столбцов.
--------------------
- В критических столбцах, как `name`, `genre` и `year_of_release`, пропуски заменить не представляется возможным (за исключением вариантов поиска данных в интернете или загрузки из библиотеки с оценками и иными данными, если такая есть), поэтому их стоит удалить. 
- Вероятно, была допущена ошибка при сборе данных.
--------------------
- Пропуски в столбце `rating` можно заменить на значение `Unknown` из-за недостатка каких-то связей для другой логической замены, кроме как создания отдельной категории.
- Некоторые игры могли не проходить сертификацию ESRB, в том числе могли быть предназначены для рынков, где эта сертификация не использовлась/не используется всё ещё, а может и вовсе не была обязательной.
--------------------
- Появившиеся пропуски в столбцах `eu_sales` и `jp_sales` можно заменить на 0 (отсутствие продаж в том или ином регионе) или на среднее значение в зависимости от названия платформы и года выхода игры. Я решил выбрать второй вариант.
- Ошибки могут быть вызваны ошибками в сборе данных или недостатком конкретной информации о продажах.
--------------------
- В столбцах `critic_score` и `user_score` с одной стороны можно заменить на индикатор -1, а можно на средние. Однако, я выберу вариант с индикатором, ибо средние значения исказят смысл данных, а сами игры могут быть просто не оценены.
- При этом возможна просто ошибка при сборе данных или просто их отсутствие (н-р, на super-mario 1985 я из интереса вообще оценок не нашёл) 
--------------------
- Исправлены типы данных:
  - `eu_sales` —> `float64`
  - `jp_sales` —> `float64`
--------------------
[Вернуться к содержанию](#Содержание)

- Обработайте пропущенные значения. Для каждого случая вы можете выбрать оптимальный, на ваш взгляд, вариант: заменить на определённое значение, оставить как есть или удалить.
- Если вы решите заменить пропуски на значение-индикатор, то убедитесь, что предложенное значение не может быть использовано в данных.
- Если вы нашли пропуски в данных с количеством проданных копий игры в том или ином регионе, их можно заменить на среднее значение в зависимости от названия платформы и года выхода игры.

In [ ]:
# Удаление пропусков в столбцах name, genre, year_of_release
df = df.dropna(subset = ['name', 'genre', 'year_of_release'])

In [ ]:
total_rows_2 = df.shape[0]
delete_1 = total_rows_1 - total_rows_2

In [ ]:
# Замена пропусков в стоблце рейтинга ESRB на новую категория unknown
df['rating'] = df['rating'].fillna('Unknown')

In [ ]:
# Замена оценок на индикатор -1
df['critic_score'] = df['critic_score'].fillna(-1)
df['user_score'] = df['user_score'].fillna(-1)

In [ ]:
# Замена продаж на средние в зависимости от названия платформы и года выхода игры
group_means = df.groupby(['platform', 'year_of_release'])[['eu_sales', 'jp_sales']].mean()

def fill_group_sales(row):
    mean_values = group_means.loc[(row['platform'], row['year_of_release'])]
    if pd.isna(row['eu_sales']):
        row['eu_sales'] = mean_values['eu_sales']
    if pd.isna(row['jp_sales']):
        row['jp_sales'] = mean_values['jp_sales']
    return row

df = df.apply(fill_group_sales, axis=1)

In [ ]:
df.isna().sum()

<a class="anchor" id="Работа-с-типом-данных-ч2"></a>
**Дополнительные преобразования данных**

In [ ]:
df['user_score'] = df['user_score'].astype('float64')
df['critic_score'] = df['critic_score'].astype('int64')
df['year_of_release'] = df['year_of_release'].astype('int64')

In [ ]:
df.isna().sum()

**Промежуточный вывод по анализу и обработке пропусков**

- Удалены пропуски по столбцам `name`, `genre`, `year_of_release` из-за их критичности этих столбцов и невозможности логической замены.
- Произведена замена в столбце `rating` на значение `Unknown` в связи с невозможностью логичной замены, а также отсутствия этого рейтинга в других странах.
- Произведена замена пропусков, образовавшихся после после преобразования типа данных в столбцах `jp_sales` и `eu_sales`. Пропуски в них были заменены на средние значения в зависимости от названия платформы и года выхода игры.
- Пропуски в столбцах `critic_score` и `user_score` заменены на индикатор `-1` из-за невозможности логической замены. Игры могут не иметь оценки критиков или пользователей. Для примера тот же марио 1985 года

---
[Вернуться к содержанию](#Содержание)

<a class="anchor" id="Работа-с-дубликатами"></a>
### 2.4. Явные и неявные дубликаты в данных

- Изучите уникальные значения в категориальных данных, например с названиями жанра игры, платформы, рейтинга и года выпуска. Проверьте, встречаются ли среди данных неявные дубликаты, связанные с опечатками или разным способом написания.
- При необходимости проведите нормализацию данных с текстовыми значениями. Названия или жанры игр можно привести к нижнему регистру, а названия рейтинга — к верхнему.

In [ ]:
df.dtypes

In [ ]:
df['name'].unique().tolist()

In [ ]:
df['platform'].unique().tolist()

In [ ]:
df['genre'].unique().tolist()

In [ ]:
# Приведение названий игра и жанра к нижнему регистру
df['name'] = df['name'].str.lower()
df['genre'] = df['genre'].str.lower()

# Приведение возрастного рейтинга к верхнему регистру
df['rating'] = df['rating'].str.upper()

In [ ]:
df['genre'].unique().tolist()

- После того как нормализуете данные и устраните неявные дубликаты, проверьте наличие явных дубликатов в данных.

In [ ]:
total_duplicates = df.duplicated().sum()
total_rows_3 = total_rows_2 - total_duplicates

In [ ]:
print(f"Всего явных дубликатов: {total_duplicates}")

In [ ]:
# Удаление явных дубликатов
df = df.drop_duplicates()

- Напишите промежуточный вывод: укажите количество найденных дубликатов и действия по их обработке.

**Промежуточный вывод:**

*Явные дубликаты:*
- 235 строк явных дубликатов.
- Явные дубликаты были удалены, так как составляют незначительную часть данных и не являются подходящими для анализа.

*Неявные дубликаты:*
- Неявные дубликаты найдены в столбце `genre`.
- Для избежания проблем с ним и другими столбцами (`name` и `raiting`) они были преобразованы в нижний и верхний регистры.
---
- Дополнительно исправлены типы данных:
  - `year_of_release` —> `int64`
  - `user_score` —> `float64`
  - `critic_score` —> `int64`

---
[Вернуться к содержанию](#Содержание)

- В процессе подготовки данных вы могли что-либо удалять, например строки с пропусками или ошибками, дубликаты и прочее. В этом случае посчитайте количество удалённых строк в абсолютном и относительном значениях.

In [ ]:
print(f"Изначально было {total_rows_1} строк")
print("------------------------------")
print(f"Удалено строк с пропущенными значеними - {delete_1}, что составляет {round(delete_1 / total_rows_1, 2)}% изначального количества")
print(f"В датасете теперь {total_rows_2} строк")
print("------------------------------")
print(f"Удалено {total_duplicates} дубликатов, что составляет {round(total_duplicates / total_rows_2, 2)}% количества до удаления")
print(f"В конечном итоге осталось {total_rows_3} строк")
print("------------------------------")
print(f"Было {total_rows_1} строк, стало {total_rows_3}")
print(f"В общем удалено {total_duplicates + delete_1}, что {round((total_duplicates + delete_1) / total_rows_1, 2)}% от всех данных")

In [ ]:
# Комментарий ревьюера
# Проверим сколько удалено строк датасета
a, b = len(tmp), len(df)
print(" Было строк в исходном датасете", a,
      '\n', "Осталось строк в датасете после обработки", b,
      '\n', "Удалено строк в датасете после обработки", a-b,
      '\n', "Процент потерь", round((a-b)/a*100, 2))

- После проведения предобработки данных напишите общий промежуточный вывод.

In [ ]:
df.columns

**Общий промежуточный вывод**
- Было удалено 235 явных дубликатов, а также исправлены неявные дубликаты в значениях столбца `genre`.
- Было удалено 277 строк с пропущенными значеними, которые были критическими, и их замена не представлялась возможной.
---
- Пропуски в `critic_score` и `user_score` заменены индикатором -1,ибо средние значения исказят смысл данных, а сами игры могут быть просто не оценены.
- Появившиеся пропуски в столбцах `eu_sales` и `jp_sales` заменены на среднее значение в зависимости от названия платформы и года выхода игры.
- Пропуски в столбце `rating` заменены на значение Unknown из-за недостатка каких-то связей для другой логической замены, кроме как создания отдельной категории.
- Значения по столбцам `name` и `genre` приведены к нижнему регистру, а значения по столбцу `rating` к верхнему регистру.
- В критических столбцах, как `name`, `genre` и `year_of_release`, пропуски были удалены.
---
- Типы данных приведены к соответствующим:
  - `year_of_release` —> `int64`
  - `eu_sales` —> `float64`
  - `jp_sales` —> `float64`
  - `user_score` —> `float64`
  - `critic_score` —> `int64`
---
[Вернуться к содержанию](#Содержание)

---
<a class="anchor" id="Фильтрация-данных"></a>
## 3. Фильтрация данных

Коллеги хотят изучить историю продаж игр в начале XXI века, и их интересует период с 2000 по 2013 год включительно. Отберите данные по этому показателю. Сохраните новый срез данных в отдельном датафрейме, например `df_actual`.

In [ ]:
# Создаем сред для интересующегося периода с 2000 по 2013
df_actual_slice = df[(df['year_of_release'] >= 2000) & (df['year_of_release'] <= 2013)]

In [ ]:
# Делаем копию смреза для избежания предупреждений и возможныъ ошибок
df_actual = df_actual_slice.copy()

In [ ]:
df_actual.info()

**Оценка полученного датасета**
- Датасет иметт 12781 строку и 11 столбцов.
- Ни в одном столбце нет пропусков.
- Типы данных полностью правильны и соотвествуют необходимым для строк.
- 5 столбцов с данными типа `float64`, 2 столбца с данными типа `int64`, 4 столбца с данными типа `object`.
---
[Вернуться к содержанию](#Содержание)

---
<a class="anchor" id="Категоризация-данных"></a>
## 4. Категоризация данных
    
Проведите категоризацию данных:
- Разделите все игры по оценкам пользователей и выделите такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала).

In [ ]:
# Функция для категоризации оценок пользователей по категориям оценок
def categorize_user_score(score):
    if 8 <= score <= 10:
        return "Высокая"
    elif 3 <= score < 8:
        return "Средняя"
    elif 0 <= score < 3:
        return "Низкая"
    elif score == -1:
        return "Нет данных"

# Применение данной функции через метод apply() для создания нового столбца с категорией по пользователям
df_actual['user_score_category'] = df_actual['user_score'].apply(categorize_user_score)

In [ ]:
df_actual.head(3)

- Разделите все игры по оценкам критиков и выделите такие категории: высокая оценка (от 80 до 100 включительно), средняя оценка (от 30 до 80, не включая правую границу интервала) и низкая оценка (от 0 до 30, не включая правую границу интервала).

In [ ]:
# Функция для категоризации оценок критиков по категориям оценок
def categorize_critic_score(score):
    if 80 <= score <= 100:
        return "Высокая"
    elif 30 <= score < 80:
        return "Средняя"
    elif 0 <= score < 30:
        return "Низкая"
    elif score == -1:
        return "Нет данных"
    
# Применение данной функции через метод apply() для создания нового столбца с категорией по критикам
df_actual['critic_score_category'] = df_actual['critic_score'].apply(categorize_critic_score)

In [ ]:
df_actual.head(3)

- После категоризации данных проверьте результат: сгруппируйте данные по выделенным категориям и посчитайте количество игр в каждой категории.

In [ ]:
# Кол-во игр в каждой категории для оценок пользователей
df_actual.groupby('user_score_category')['name'].count()

In [ ]:
# Кол-во игр в каждой категории для оценок критиков
df_actual.groupby('critic_score_category')['name'].count()

- Выделите топ-7 платформ по количеству игр, выпущенных за весь актуальный период.

In [ ]:
# Группировка по платформам для создания топ-7 платформ по кол-ву игр
top_platfroms_by_games = df_actual.groupby('platform', as_index=False)['name'].count()
top_platfroms_by_games.sort_values(by='name', ascending=False, inplace=True)
top_platfroms_by_games.rename(columns={'name': 'total_games'}, inplace=True)

# Изменение индекса для лучшей визуализации
top_platfroms_by_games = top_platfroms_by_games.reset_index(drop=True)
top_platfroms_by_games.index = range(1, len(top_platfroms_by_games) + 1)

In [ ]:
print("Топ-7 платформ по количеству игр за актуальный период:")
top_7_platforms_by_games = top_platfroms_by_games.head(7)
top_7_platforms_by_games

[Вернуться к содержанию](#Содержание)

---
<a class="anchor" id="Итоговый-вывод"></a>
## 5. Итоговый вывод

В конце напишите основной вывод и отразите, какую работу проделали. Не забудьте указать описание среза данных и новых полей, которые добавили в исходный датасет.

### Краткое описание целей и задач

- Отобрать данные по времени выхода игры за период с 2000 по 2013 год включительно.
- Категоризовать игры по оценкам пользователей и экспертов. Выделите три категории:
  - высокая оценка — с оценкой от 8 до 10 и от 80 до 100, включая правые границы интервалов.
  - средняя оценка — с оценкой от 3 до 8 и от 30 до 80, не включая правые границы интервалов.
  - низкая оценка — с оценкой от 0 до 3 и от 0 до 30, не включая правые границы интервалов.
- Выделить топ-7 платформ по количеству игр, выпущенных за весь требуемый период.

### Основные результаты, полученные в ходе выполнения проекта

1. Обработаны данные: типы данных, пропуски, дубликаты и т.д.
2. Получен срез по времени выхода игры за период с 2000 по 2013 год включительно.
3. Произведена категоризация по оценкам пользователей и критиков на три категории: высокая, средняя, низкая.
4. Выделены топ-7 платформ по количеству игр, выпущенных за весь требуемый период.

### Анализ разделов проекта

**Загрузка данных и знакомство с ними**
- Данные загружены и проанализированы.

**Проверка ошибок в данных и их предобработка**
- Столбцы приведены к формату `snake_case`.
- Значения столбцов `name` и `genre` к нижнему регистру, значения столбца `raiting` к верхнему.
---
- Обработаны типы данных:
  - `year_of_release` —> `int64`
  - `eu_sales` —> `float64`
  - `jp_sales` —> `float64`
  - `user_score` —> `float64`
  - `Critic score` —> `int64`
---
- Было удалено 235 явных дубликатов, а также исправлены неявные дубликаты в значениях столбца `genre`.
- Было удалено 277 строк с пропущенными значеними, которые были критическими, и их замена не представлялась возможной. Это качается столбцов `name`, `genre`, `year_of_release`.
- Пропуски в `critic_score` и `user_score` заменены индикатором -1,ибо средние значения исказят смысл данных, а сами игры могут быть просто не оценены.
- Появившиеся пропуски в столбцах `eu_sales` и `jp_sales` заменены на среднее значение в зависимости от названия платформы и года выхода игры.
- Пропуски в столбце `rating` заменены на значение `UNKNOWN` из-за недостатка каких-то связей для другой логической замены, кроме как создания отдельной категории.

**Фильтрация данных**
- Получен срез данных `df_actual` с периодом с 2000 по 2013 года.
- Создана копия и проведен анализ нового датасета/среза.

**Категоризация данных**
- По этому срезу произведена категоризация по столбцам `critic_score` и `user_score` и созданы новые столбцы `user_score_category` и `critic_score_category` с категориями оценок: высокая, средняя, низкая.
- Создан датафрейм `top_7_platforms_by_games` с топ-7 платформами по количеству игр на них.
- Дополнительно индексы приведены к месту в топ-7 для красоты и лучшего визуального понимания и восприятия.

### Возможные направления дальнейшего развития и применения полученных результатов

- После обработки данных и получения среза можно выполнить необходимые исследования, которые планировались командой игры "Секреты Темнолесья".
- Для примера можно рассмотреть самые высокооцененные жанры, в какие годы были популярны какие-либо жанры и так далее.
- В добавок, можно по категоризации провести дополнительный анализ.
- Можно проанализировать самые популярные платформы по таблице топ-7 платформ по количеству игр на них.
---
[Вернуться к содержанию](#Содержание)